(Visit the
[documentation](https://datafold-dev.gitlab.io/datafold/tutorial_index.html) page
to view the executed notebook.)

# Extended Dynamic Mode Decomposition on Limit Cycle

In this tutorial, we explore the (Extended-) Dynamic Mode Decomposition (E-DMD). We set up a non-linear ordinary differential equation (ODE) system, generate time series data with it and model the dynamics with an `EDMD` model. 

Note that all models for time series modelling require a `TSCDataFrame` type to fit a model. The initial conditions for the `predict` method can be either a `numpy.ndarray`, a `pandas.DataFrame`, or in some circumstances (when multiple samples are required to define an initial condition) a `TSCDataFrame`.

In [2]:
!pip install numexpr

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 471 kB 12.2 MB/s eta 0:00:01
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp

from datafold.appfold import EDMD
from datafold.dynfold import DMDFull
from datafold.dynfold.transform import TSCPolynomialFeatures, TSCRadialBasis
from datafold.pcfold import GaussianKernel, TSCDataFrame

## Set up ODE system

We set up a Hopf ODE system:

$$
\dot{y}_0 = -y_1 + y_0 (\mu - y_0^2 - y_1^2) \\
\dot{y}_1 = y_0 + y_1 (\mu - y_0^2 - y_1^2)
$$

with $\mu=1$. The ODE system has an circle shaped attractor which is centered at the origin. All sampled initial conditions are off the attractor (i.e. the time series are sampled on the transient phase space region). 

We solve the system by integration with a Runge-Kutta45 scheme using scipy's ODE solver. The return type of this function is a `TSCDataFrame` and includes the time series for each initial condition (a row in argument `initial_conditions`).

In [5]:
def solve_limit_cycle(initial_conditions, t_eval):
    def limit_cycle(t, y):
        """ODE system."""
        mu = 1
        y_dot = np.zeros(2)

        factor = mu - y[0] ** 2 - y[1] ** 2

        y_dot[0] = -y[1] + y[0] * factor
        y_dot[1] = y[0] + y[1] * factor
        return y_dot

    assert initial_conditions.ndim == 2
    assert initial_conditions.shape[1] == 2

    time_series_dfs = []

    for ic in initial_conditions:
        solution = solve_ivp(
            limit_cycle, t_span=(t_eval[0], t_eval[-1]), y0=ic, t_eval=t_eval
        )

        solution = pd.DataFrame(
            data=solution["y"].T,
            index=solution["t"],
            columns=["x1", "x2"],
        )

        time_series_dfs.append(solution)

    return TSCDataFrame.from_frame_list(time_series_dfs)

In [18]:
from load_data import load_patient_trial
data_p1, labels_p1, data_p2, labels_p2 = load_patient_trial()
labels_p1 = np.array(labels_p1)
labels_p2 = np.array(labels_p2)

Creating RawArray with float64 data, n_channels=9, n_times=57728
    Range : 0 ... 57727 =      0.000 ...   225.496 secs
Ready.
40 events found
Event IDs: [0 1]
20 events found
Event IDs: [1 2 3 4]
Not setting metadata
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Loading data for 20 events and 1884 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=9, n_times=58112
    Range : 0 ... 58111 =      0.000 ...   226.996 secs
Ready.
40 events found
Event IDs: [0 1]
20 events found
Event IDs: [1 2 3 4]
Not setting metadata
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Loading data for 20 events and 1884 original time points ...
0 bad epochs dropped


## Sampling the dynamical system

We now start collecting time series data from the Hopf system (our training set). To sample the phase space, we systematically distribute initial conditions and solve the ODE system for rather short time intervals.

## 1. DMD: Identity dictionary

In our first model, we use a Dynamic Mode Decomposition (in `datafold.dynfold.dmd`) model and decompose the data in spatio-temporal coordinates using the original form of the time series. In other words, our dictionary only includes the state identities "x1" and "x2" as observable functions. 

In the first attempt, we use the `DMDFull` model directly. The same could be accomplished with `EDMD(dict_step=["id", TSCIdentity()]`).

Note that the DMD-based models' API aligns with scikit-learn. However, the input type of `X` is restricted to a `TSCDataFrame`. The `predict` method allows setting an array of `time_values`, where we can choose at which time samples to evaluate the model. In our case, we are interested in reconstructing the training data, we leave the parameter `time_values=None`. The model then uses the same time values that were available during `fit`.

In [40]:
tsc_data = TSCDataFrame.from_frame_list([pd.DataFrame(data_p1[0,:,:].T), pd.DataFrame(data_p1[1,:,:].T),
                                        pd.DataFrame(data_p1[2,:,:].T)])

In [41]:
tsc_data.initial_states()

,feature,0,1,2,3,4,5,6,7
ID,time,,,,,,,,
0,0,-0.000003,-4.279149e-06,-0.000002,-0.000004,-0.000005,-0.000001,6.245087e-07,-0.000002
1,0,0.000005,7.467624e-06,0.000007,0.000002,0.000005,0.000004,3.517393e-06,0.000003
2,0,0.000003,-2.135298e-07,-0.000003,-0.000005,-0.000003,0.000001,-2.586921e-06,-0.000002


In [42]:
dmd = DMDFull().fit(X=tsc_data, store_system_matrix=True)  # must be TSCDataFrame
dmd_values = dmd.predict(tsc_data.initial_states(), time_values=None)

# Will be a red line in the plot
#dmd_values_oos = dmd.predict(np.array([-1.8, 2, 3,4,  5, 6, 7, 8]), time_values=np.linspace(0, 100, 1000))

print("Data snipped with predicted time series data")
dmd_values

Data snipped with predicted time series data


feature             0             1             2             3             4  \
ID time                                                                         
0  0    -2.809587e-06 -4.279149e-06 -2.304777e-06 -3.910479e-06 -5.283649e-06   
   1    -2.622089e-06 -3.772948e-06 -2.022790e-06 -3.542879e-06 -4.798003e-06   
   2    -2.439795e-06 -3.332155e-06 -1.769695e-06 -3.201049e-06 -4.352875e-06   
   3    -2.264383e-06 -2.947531e-06 -1.542927e-06 -2.884453e-06 -3.945254e-06   
   4    -2.096984e-06 -2.611250e-06 -1.340103e-06 -2.592278e-06 -3.572315e-06   
...               ...           ...           ...           ...           ...   
2  1879 -3.996605e-62 -6.153112e-62 -9.788610e-62 -1.121417e-61 -7.618160e-62   
   1880 -3.731654e-62 -5.745197e-62 -9.139683e-62 -1.047073e-61 -7.113121e-62   
   1881 -3.484267e-62 -5.364324e-62 -8.533776e-62 -9.776586e-62 -6.641563e-62   
   1882 -3.253281e-62 -5.008701e-62 -7.968036e-62 -9.128456e-62 -6.201266e-62   
   1883 -3.037608e-62 -4.676654e-62 -7.439803e-62 -8.523293e-62 -5.790159e-62   

feature             5             6             7  
ID time                                            
0  0    -1.404258e-06  6.245087e-07 -2.351405e-06  
   1    -1.316600e-06  6.557466e-07 -2.055231e-06  
   2    -1.229206e-06  6.772350e-07 -1.789244e-06  
   3    -1.143547e-06  6.902362e-07 -1.551008e-06  
   4    -1.060679e-06  6.959093e-07 -1.338195e-06  
...               ...           ...           ...  
2  1879 -3.860951e-62 -6.718720e-62 -4.734396e-62  
   1880 -3.604992e-62 -6.273308e-62 -4.420534e-62  
   1881 -3.366003e-62 -5.857425e-62 -4.127478e-62  
   1882 -3.142856e-62 -5.469112e-62 -3.853851e-62  
   1883 -2.934504e-62 -5.106542e-62 -3.598363e-62  

[5652 rows x 8 columns]

### Compare with training data 

We can now compare the original time series data with the data-driven reconstruction of the DMD model. From what we see in the plots below is that the DMD model performs poorly. This is not surprising at this stage, because we learn the Koopman matrix directly on the available states. The computed Koopman matrix is therefore a $K \in \mathbb{R}^{[2 \times 2]}$ describing a linear system

$$ x_{n+1} = K x_n $$

and not being able to desribe a complex dynamics such as this of the underlying system. Note that the learnt system equation implies that we have modelled a dicrete system, while the underling system is continuous. This is a result from the discretely sampled data with a fixed time interval. Because we are in this easier setting of a 2-by-2 matrix, in the next cell, we look at the relation to a continuous system.

In [43]:
generator_A = (dmd.koopman_matrix_ - np.eye(8)) / dmd.dt_

det = np.linalg.det(generator_A)
trace = np.trace(generator_A)

print("Relevant values for the stability analysis: \n")
print(f"determinant of A: {det}")
print(f"trace of A: {trace}")

print(f"Delta {1/4. * trace ** 2} ")

Relevant values for the stability analysis: 

determinant of A: 9.604063625417282e-09
trace of A: -0.8220860831563211
Delta 0.16895638202982544 


## 2. EDMD: Polynomial feature dictionary

We now get to the "extended" part of a Dynamic Model Decomposition: We define a *dictionary* in which we process the time series data before we fit a DMD model with it. For this, we use the `datafold.appfold.EDMD` class, which is a [`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html?highlight=pipeline#sklearn.pipeline.Pipeline). In the `EDMD` model, a dictionary can be a flexible number of transform models that are process the time series data consecutively (in the same order as defined). The final estimator has to be a `datafold.dynfold.dmd.DMDBase` model and defaults to `DMDFull`.  

Choosing the "right" dictionary is not an easy task and is similar to "model selection" in classical machine learning. In our choice of dictionary, we can include expert knowledge, e.g. if we know the principle equations from an underlying physical system from which time series are collected. We can also apply methods from functional theory to represent the data in another basis to linearize the unknown dynamics manifold. 

In the first dictionary, we use `TSCPolynomialFeatures` which is a wrapper of [`sklearn.preprocessing.PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn.preprocessing.PolynomialFeatures) to support `TSCDataFrame` type.

In [44]:
tsc_data.shape

(5652, 8)

In [45]:
dict_step = [
    (
        "polynomial",
        TSCPolynomialFeatures(degree=3),
    )
]

edmd_poly = EDMD(dict_steps=dict_step, include_id_state=True).fit(X=tsc_data)
edmd_poly_values = edmd_poly.predict(tsc_data.initial_states())

/home/jupyter-sazeglio/SSVEP_IEEE_SMC_2021/Code/datafold-master/datafold/dynfold/dmd.py:1192: UserWarning: Shift matrix (shape=(164, 164)) has not full rank (=34), falling back to least squares solution. The sum of residuals is: 0.0
  f"Shift matrix (shape={G.shape}) has not full rank (={rank}), falling "


### Analyze the dictionary

Before we compare the model's time series data to the training data, we investigate how we to analyze the actual process of dictionary transformations in an `EDMD` model.  

This is useful if we are interested and want to investigate the values of the "dictionary space", i.e. the data representation after the transformations were applied to the original data and before it is passed to the final DMD model. To accomblish this we can use the `transform` method of `EDMD`, which only applies the dictionary transformations without processing it through the final estimator. 

In the following cell, we see that the result is a `TSCDataFrame`, which includes the original states "x1" and "x2" plus the generated polynomial features. 

The single dictionary models are accessible with the specified name via `named_steps`. Here, we access the model and its attribute `TSCPolynomialFeatures.powers_` through the `EDMD` model.

In [46]:
# access models in the dictionary, the name was given in "dict_step" above
print(edmd_poly.named_steps["polynomial"])

print("")
print("polynomial degrees for data (first column 'x1' and second 'x2'):")
print(edmd_poly.named_steps["polynomial"].powers_)

print("")
print("Dictionary space values:")
edmd_poly.transform(tsc_data)

TSCPolynomialFeatures(degree=3)

polynomial degrees for data (first column 'x1' and second 'x2'):
[[2 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 2 1]
 [0 0 0 ... 0 1 2]
 [0 0 0 ... 0 0 3]]

Dictionary space values:


feature             0             1             2         3             4  \
ID time                                                                     
0  0    -2.809587e-06 -4.279149e-06 -2.304777e-06 -0.000004 -5.283649e-06   
   1     5.975949e-07 -2.426259e-06 -6.244451e-07 -0.000002 -2.796224e-06   
   2     2.173631e-06 -1.202320e-06  1.476857e-06  0.000002  8.834052e-07   
   3    -3.283235e-07 -2.152595e-06  1.175581e-06  0.000003  3.156872e-06   
   4    -2.956322e-06 -2.803809e-06 -7.604847e-08  0.000003  3.062731e-06   
...               ...           ...           ...       ...           ...   
2  1879  3.512094e-06  2.887042e-06  4.651117e-06  0.000004  4.989297e-06   
   1880  1.304314e-06  1.574965e-06  2.930849e-06  0.000002  3.328418e-06   
   1881  6.420650e-07  1.741788e-06  3.013410e-06  0.000003  2.604061e-06   
   1882  8.445442e-07  1.201872e-06  3.704851e-06  0.000005  3.495549e-06   
   1883  6.971399e-07 -1.257549e-07  3.633067e-06  0.000007  4.576515e-06   

feature             5             6             7           0^2           0 1  \
ID time                                                                         
0  0    -1.404258e-06  6.245087e-07 -2.351405e-06  7.893780e-12  1.202264e-11   
   1     7.402672e-07  1.980965e-06 -2.256284e-07  3.571197e-13 -1.449920e-12   
   2     2.631692e-06  3.790763e-06  2.865402e-06  4.724673e-12 -2.613399e-12   
   3     1.180399e-06  3.292876e-06  3.481240e-06  1.077963e-13  7.067477e-13   
   4    -2.147842e-06  1.405774e-06  1.991534e-06  8.739842e-12  8.288963e-12   
...               ...           ...           ...           ...           ...   
2  1879  3.399323e-06  1.120383e-06  3.300293e-06  1.233480e-11  1.013956e-11   
   1880  1.060477e-06  8.164172e-07  2.655035e-06  1.701235e-12  2.054248e-12   
   1881  1.867726e-08  1.705589e-06  3.648817e-06  4.122475e-13  1.118341e-12   
   1882 -2.593598e-07  2.028225e-06  4.120880e-06  7.132550e-13  1.015034e-12   
   1883 -4.243764e-07  1.068312e-06  3.365337e-06  4.860041e-13 -8.766877e-14   

feature  ...           5^3         5^2 6         5^2 7         5 6^2  \
ID time  ...                                                           
0  0     ... -2.769113e-18  1.231494e-18 -4.636830e-18 -5.476762e-19   
   1     ...  4.056630e-19  1.085560e-18 -1.236433e-19  2.904972e-18   
   2     ...  1.822658e-17  2.625407e-17  1.984521e-17  3.781711e-17   
   3     ...  1.644699e-18  4.588102e-18  4.850556e-18  1.279911e-17   
   4     ... -9.908481e-18  6.485154e-18  9.187395e-18 -4.244568e-18   
...      ...           ...           ...           ...           ...   
2  1879  ...  3.928051e-17  1.294647e-17  3.813619e-17  4.267026e-18   
   1880  ...  1.192625e-18  9.181522e-19  2.985882e-18  7.068472e-19   
   1881  ...  6.515373e-24  5.949776e-22  1.272853e-21  5.433278e-20   
   1882  ... -1.744648e-20  1.364336e-19  2.772013e-19 -1.066927e-18   
   1883  ... -7.642821e-20  1.923979e-19  6.060815e-19 -4.843364e-19   

feature         5 6 7         5 7^2           6^3         6^2 7         6 7^2  \
ID time                                                                         
0  0     2.062115e-18 -7.764291e-18  2.435653e-19 -9.170741e-19  3.452975e-18   
   1    -3.308712e-19  3.768564e-20  7.773744e-18 -8.854157e-19  1.008473e-19   
   2     2.858560e-17  2.160759e-17  5.447282e-17  4.117550e-17  3.112418e-17   
   3     1.353126e-17  1.430529e-17  3.570477e-17  3.774720e-17  3.990647e-17   
   4    -6.013199e-18 -8.518786e-18  2.778092e-18  3.935671e-18  5.575591e-18   
...               ...           ...           ...           ...           ...   
2  1879  1.256931e-17  3.702520e-17  1.406369e-18  4.142719e-18  1.220314e-17   
   1880  2.298707e-18  7.475524e-18  5.441723e-19  1.769679e-18  5.755095e-18   
   1881  1.162357e-19  2.486665e-19  4.961617e-18  1.061453e-17  2.270799e-17   
   1882 -2.167747e-18 -4.404358e-18  8.343499e-18  1.695205e-17  3.444261e-17   
   1883 -1.525730e-18 -4.806273e-18

## 3. EDMD: Radial basis function dictionary

In our last attempt, we set up a dictionary with `TSCRadialBasis`. The transform class computes coefficients of each time series sample to a set of radial basis functions, which centres' are distributed on the phase space. The radial basis functions, therefore, provide a way to linearize the phase space's manifold. Here we choose a Gaussian kernel and set the centre of the functions to the initial condition states.

In the time series in "dictionary space," we see that the feature dimension is now much greater than at the beginning (i.e. we provide a larger set of observables to compute the Koopman operator).

In [47]:
dict_step = [
    (
        "rbf",
        TSCRadialBasis(
            kernel=GaussianKernel(epsilon=0.17), center_type="initial_condition"
        ),
    )
]

edmd_rbf = EDMD(dict_steps=dict_step, include_id_state=True).fit(
    X=tsc_data
)  # Note that the "extended" part is in the transformations
edmd_rbf_values = edmd_rbf.predict(tsc_data.initial_states())

len_koopman_matrix = len(edmd_rbf.named_steps["dmd"].eigenvectors_right_)
print(f"shape of Koopman matrix: {len_koopman_matrix} x {len_koopman_matrix}")
edmd_rbf.transform(tsc_data)

shape of Koopman matrix: 11 x 11


/home/jupyter-sazeglio/SSVEP_IEEE_SMC_2021/Code/datafold-master/datafold/dynfold/dmd.py:1192: UserWarning: Shift matrix (shape=(11, 11)) has not full rank (=9), falling back to least squares solution. The sum of residuals is: 0.0
  f"Shift matrix (shape={G.shape}) has not full rank (={rank}), falling "


feature             0             1             2         3             4  \
ID time                                                                     
0  0    -2.809587e-06 -4.279149e-06 -2.304777e-06 -0.000004 -5.283649e-06   
   1     5.975949e-07 -2.426259e-06 -6.244451e-07 -0.000002 -2.796224e-06   
   2     2.173631e-06 -1.202320e-06  1.476857e-06  0.000002  8.834052e-07   
   3    -3.283235e-07 -2.152595e-06  1.175581e-06  0.000003  3.156872e-06   
   4    -2.956322e-06 -2.803809e-06 -7.604847e-08  0.000003  3.062731e-06   
...               ...           ...           ...       ...           ...   
2  1879  3.512094e-06  2.887042e-06  4.651117e-06  0.000004  4.989297e-06   
   1880  1.304314e-06  1.574965e-06  2.930849e-06  0.000002  3.328418e-06   
   1881  6.420650e-07  1.741788e-06  3.013410e-06  0.000003  2.604061e-06   
   1882  8.445442e-07  1.201872e-06  3.704851e-06  0.000005  3.495549e-06   
   1883  6.971399e-07 -1.257549e-07  3.633067e-06  0.000007  4.576515e-06   

feature             5             6             7  rbf0  rbf1  rbf2  
ID time                                                              
0  0    -1.404258e-06  6.245087e-07 -2.351405e-06   1.0   1.0   1.0  
   1     7.402672e-07  1.980965e-06 -2.256284e-07   1.0   1.0   1.0  
   2     2.631692e-06  3.790763e-06  2.865402e-06   1.0   1.0   1.0  
   3     1.180399e-06  3.292876e-06  3.481240e-06   1.0   1.0   1.0  
   4    -2.147842e-06  1.405774e-06  1.991534e-06   1.0   1.0   1.0  
...               ...           ...           ...   ...   ...   ...  
2  1879  3.399323e-06  1.120383e-06  3.300293e-06   1.0   1.0   1.0  
   1880  1.060477e-06  8.164172e-07  2.655035e-06   1.0   1.0   1.0  
   1881  1.867726e-08  1.705589e-06  3.648817e-06   1.0   1.0   1.0  
   1882 -2.593598e-07  2.028225e-06  4.120880e-06   1.0   1.0   1.0  
   1883 -4.243764e-07  1.068312e-06  3.365337e-06   1.0   1.0   1.0  

[5652 rows x 11 columns]

### Compare with training data

Again for comparison, we plot the training time series next to the EDMD model's time series. This time the phase portraits match quite well. However, at this stage, this is only an indicator of a successful model. Like for all data-driven machine learning models, there is always the danger of overfitting the training data. A consequence would be a poor generalization for out-of-sample initial conditions. 

The right way to tackle overfitting is to apply cross-validation. For the `EDMD` model this can be achieved with `EDMDCV`, which allows an exhaustive search over a grid of the model's and the dictionary model parameters. *datafold* provides time series splitting for cross-validation which enables measuring the model's quality on unseen (partial) time series data.

In this tutorial, we only add a single out-of-sample initial condition and compare it to the ODE system for a longer time series as in the training data. We used this plot to visually "optimize" the Gaussian kernel epsilon value. If we now predict the time series we want to highlight that the `EDMD` model interpolates in time. This means we are now able to freely choose the time interval and number of time samples at which to evaluate the model. In the time series we can see that the model follows the ground truth solution fairly well for some time. However, the `EDMD` model won't stay on the attractor for $t \rightarrow \infty$ yet.

The problem of overfitting can be seen if `epsilon=1` is set in the Gaussian kernel. The reconstruction phase portrait looks equally well, but the out-of-sample quality decreases. 